# Belgian pharmacies

In [1]:
from datetime import datetime
import orjson

def read_json(path):
    with open(str(path), encoding="utf8") as file:
        return orjson.loads(file.read())

stats = read_json("stats.json")
pharmacies = read_json("last-pharmacies_enhancedVersion.json")

REGIONS = ["Brussels", "Flanders", "Wallonia"]
# sort stats ASC
convert_2_date = lambda x : datetime.strptime(x, "%d-%m-%Y")
ENTRIES = sorted( [ ( convert_2_date(key), value ) for key, value in stats.items()], key=lambda x: x[0])

## Map of pharmacies (AFMPS)

In [2]:
from ipyleaflet import Map, Marker, MarkerCluster, AwesomeIcon, FullScreenControl, SearchControl
from ipywidgets import HTML

center = (50.8503396, 4.3517103) # Brussels
m = Map(center=center, zoom=10)

# Icons
icon_active = AwesomeIcon(name='house-chimney-medical', marker_color='blue', icon_color='darkgreen')
icon_closed = AwesomeIcon(name='house-medical-circle-xmark', marker_color='red', icon_color='darkgreen')

# To detect which languages to use for opening hours
between = lambda zipCode, start, end : start <= zipCode <= end
BXL = lambda zipCode : between(zipCode, 1000, 1299)
GER = lambda zipCode : between(zipCode, 4700, 4791)
VL = lambda zipCode : between(zipCode, 1500, 3999) or between(zipCode, 8000, 9999)
def which_language(zipCode):
  if BXL(zipCode):
    return ["fr", "nl"]
  elif VL(zipCode):
    return ["nl"]
  elif GER(zipCode):
    return ["de"]
  else:
    return ["fr"]

# As my output always provide the opening hours in the 3 languages, at least one match
def generate_opening_hours_table(zipCode, opening_hours):
  table = "<table><tr><th>Opening hours</th><th></th></tr>"
  languages = which_language(zipCode)
  for row in opening_hours:
    if row["lang"] in languages:
      table += "<tr><td>{}</td><td>{}</td></tr>".format(row["lang"], row["value"])
  table += "</table>"
  return table

def generate_contact_table(contact_dict):
  table = "<table><tr><th>Means of contact</th><th></th></tr>"
  for key, val in contact_dict.items():
    table += "<tr><td>{}</td><td>{}</td></tr>".format(key, " ".join(val))
  table += "</table>"
  return table

def create_popup(pharmacy):
  common_val = "<b>{}</b><br>{}<br>{} {}".format(pharmacy["name"], pharmacy["textual_address"], pharmacy["zipCode"], pharmacy["municipality"])
  opening_hours = generate_opening_hours_table(pharmacy["zipCode"],pharmacy["opening_hours"]) if "opening_hours" in pharmacy and len(pharmacy["opening_hours"]) > 1 else ""
  contacts = generate_contact_table(pharmacy["contact"]) if "contact" in pharmacy and len(pharmacy["contact"]) > 0 else ""
  value = "{}{}{}".format(common_val, opening_hours, contacts)

  return HTML(
    value=value,
    placeholder = "",
    description = ""
  )

def create_marker(pharmacy):
  # TODO later change that trade-off
  coords = pharmacy["geo"][1]
  # constructor
  return Marker(
     icon = icon_active if pharmacy["status"].casefold() == "active" else icon_closed,
     title = pharmacy["name"],
     alt = pharmacy["name"],
     location = (coords["latitude"], coords["longitude"]),
     draggable = False,
     # Text when clicked on marker
     popup = create_popup(pharmacy)
  )

# Create markers
markers = [create_marker(pharmacy) for pharmacy in pharmacies]

marker_cluster = MarkerCluster(markers=markers)
m.add_layer(marker_cluster)
m.add_control(FullScreenControl())
m.add_control(SearchControl(
  position="topleft",
  url='https://nominatim.openstreetmap.org/search?format=json&q={s}',
  zoom=12,
  marker=Marker(icon=AwesomeIcon(name="crosshairs", marker_color='green', icon_color='darkgreen'), draggable=False)
))

m

Map(center=[50.8503396, 4.3517103], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Pharmacies by region (most recent data only)

In [ ]:
import matplotlib.pyplot as plt

most_recent = ENTRIES[-1]

labels = REGIONS
active_pharmacies = [ most_recent[1][region]["active_pharmacies"] for region in REGIONS]
inactive_pharmacies = [ most_recent[1][region]["temporarily_suspended_pharmacies"] for region in REGIONS]

fig, ax = plt.subplots()

ax.bar(labels, active_pharmacies, label='Active pharmacies')
ax.bar(labels, inactive_pharmacies, bottom=active_pharmacies, label='Temporarily suspended pharmacies')

ax.set_ylabel('Number of pharmacies')
ax.set_title('Pharmacies by region (most recent data only)')
ax.legend()

plt.show()